In [275]:
import pandas as pd

file_path = "C:/Users/info/Desktop/project3team11/Resources/U.S._Chronic_Disease_Indicators.csv"
data = pd.read_csv(file_path)


# Display the first few rows to understand the structure
print(data.head())

   YearStart  YearEnd LocationAbbr LocationDesc DataSource     Topic  \
0       2019     2019           AR     Arkansas      BRFSS  Diabetes   
1       2019     2019           ID        Idaho      BRFSS  Diabetes   
2       2019     2019           IN      Indiana      YRBSS     Sleep   
3       2019     2019           IA         Iowa       NVSS    Asthma   
4       2019     2019           IA         Iowa      BRFSS    Asthma   

                                            Question  Response DataValueUnit  \
0                              Diabetes among adults       NaN             %   
1                              Diabetes among adults       NaN             %   
2    Short sleep duration among high school students       NaN             %   
3  Asthma mortality among all people, underlying ...       NaN        Number   
4                        Current asthma among adults       NaN             %   

      DataValueType  ...  TopicID  QuestionID ResponseID DataValueTypeID  \
0  Crude P

In [276]:
# Drop irrelevant columns
columns_to_drop = [
    'Response', 'DataValueFootnoteSymbol', 'DataValueFootnote', 'StratificationCategoryID2',  
    'StratificationCategory2', 'Stratification2', 'StratificationCategory3', 
    'Stratification3', 'ResponseID', 'StratificationID2', 
    'StratificationCategoryID3', 'StratificationID3', 'LocationAbbr', 'YearStart'
]
data_cleaned = data.drop(columns=columns_to_drop)

print (data_cleaned)

        YearEnd    LocationDesc DataSource  \
0          2019        Arkansas      BRFSS   
1          2019           Idaho      BRFSS   
2          2019         Indiana      YRBSS   
3          2019            Iowa       NVSS   
4          2019            Iowa      BRFSS   
...         ...             ...        ...   
309210     2022  Virgin Islands      BRFSS   
309211     2022   West Virginia      BRFSS   
309212     2022       Wisconsin      BRFSS   
309213     2022         Vermont      BRFSS   
309214     2022      Washington      BRFSS   

                                        Topic  \
0                                    Diabetes   
1                                    Diabetes   
2                                       Sleep   
3                                      Asthma   
4                                      Asthma   
...                                       ...   
309210                                Tobacco   
309211  Chronic Obstructive Pulmonary Disease   
309212

In [277]:
# Rename columns for clarity
data_cleaned = data_cleaned.rename(columns={
    'LocationDesc': 'Location',
    'DataValue': 'Value', 
    'YearEnd' : 'Year'
})

print (data_cleaned)


        Year        Location DataSource  \
0       2019        Arkansas      BRFSS   
1       2019           Idaho      BRFSS   
2       2019         Indiana      YRBSS   
3       2019            Iowa       NVSS   
4       2019            Iowa      BRFSS   
...      ...             ...        ...   
309210  2022  Virgin Islands      BRFSS   
309211  2022   West Virginia      BRFSS   
309212  2022       Wisconsin      BRFSS   
309213  2022         Vermont      BRFSS   
309214  2022      Washington      BRFSS   

                                        Topic  \
0                                    Diabetes   
1                                    Diabetes   
2                                       Sleep   
3                                      Asthma   
4                                      Asthma   
...                                       ...   
309210                                Tobacco   
309211  Chronic Obstructive Pulmonary Disease   
309212                           Immunizat

In [278]:
# Ensure you are working with a copy, not a view
data_cleaned = data_cleaned.copy()

# Extract latitude and longitude from the Geolocation column
data_cleaned[['Longitude', 'Latitude']] = data_cleaned['Geolocation'].str.extract(r'POINT \((-?\d+\.\d+) (-?\d+\.\d+)\)')

# Convert the Latitude and Longitude columns to numeric types
data_cleaned['Latitude'] = pd.to_numeric(data_cleaned['Latitude'], errors='coerce')
data_cleaned['Longitude'] = pd.to_numeric(data_cleaned['Longitude'], errors='coerce')

# Verify the extracted columns
print(data_cleaned[['Geolocation', 'Latitude', 'Longitude']].head())


                                     Geolocation   Latitude   Longitude
0   POINT (-92.27449074299966 34.74865012400045)  34.748650  -92.274491
1  POINT (-114.3637300419997 43.682630005000476)  43.682630 -114.363730
2  POINT (-86.14996019399968 39.766910452000445)  39.766910  -86.149960
3   POINT (-93.81649055599968 42.46940091300047)  42.469401  -93.816491
4   POINT (-93.81649055599968 42.46940091300047)  42.469401  -93.816491


In [279]:
# Display the first few rows of the dataset
print(data_cleaned)

        Year        Location DataSource  \
0       2019        Arkansas      BRFSS   
1       2019           Idaho      BRFSS   
2       2019         Indiana      YRBSS   
3       2019            Iowa       NVSS   
4       2019            Iowa      BRFSS   
...      ...             ...        ...   
309210  2022  Virgin Islands      BRFSS   
309211  2022   West Virginia      BRFSS   
309212  2022       Wisconsin      BRFSS   
309213  2022         Vermont      BRFSS   
309214  2022      Washington      BRFSS   

                                        Topic  \
0                                    Diabetes   
1                                    Diabetes   
2                                       Sleep   
3                                      Asthma   
4                                      Asthma   
...                                       ...   
309210                                Tobacco   
309211  Chronic Obstructive Pulmonary Disease   
309212                           Immunizat

In [280]:
# Check unique values for Stratification columns
print(data_cleaned['Stratification1'].unique())
print(data_cleaned['StratificationCategoryID1'].unique())
print(data_cleaned['StratificationCategory1'].unique())


['Male' 'Grade 12' 'Overall' 'Age 18-44' 'Age 0-44' 'Female'
 'Multiracial, non-Hispanic' 'Age 45-64' 'Hispanic' 'Age >=65' 'Grade 11'
 'White, non-Hispanic' 'Black, non-Hispanic'
 'American Indian or Alaska Native, non-Hispanic'
 'Asian or Pacific Islander, non-Hispanic'
 'Hawaiian or Pacific Islander, non-Hispanic' 'Asian, non-Hispanic'
 'Age 4 m - 5 y' 'Age 6-14' 'Grade 9' 'Age 1-5' 'Age 12-17' 'Age 6-9'
 'Age 10-13' 'Age 6-11' 'Grade 10']
['SEX' 'GRADE' 'OVERALL' 'AGE' 'RACE']
['Sex' 'Grade' 'Overall' 'Age' 'Race/Ethnicity']


In [281]:
# Create new columns based on StratificationCategoryID1
data_cleaned['Sex'] = data_cleaned.apply(
    lambda row: row['Stratification1'] if row['StratificationCategoryID1'] == 'SEX' else None, axis=1
)

data_cleaned['Age'] = data_cleaned.apply(
    lambda row: row['Stratification1'] if row['StratificationCategoryID1'] == 'AGE' else None, axis=1
)

data_cleaned['Education'] = data_cleaned.apply(
    lambda row: row['Stratification1'] if row['StratificationCategoryID1'] == 'GRADE' else None, axis=1
)

data_cleaned['Race/Ethnicity'] = data_cleaned.apply(
    lambda row: row['Stratification1'] if row['StratificationCategoryID1'] == 'RACE' else None, axis=1
)

data_cleaned['Overall'] = data_cleaned.apply(
    lambda row: row['Stratification1'] if row['StratificationCategoryID1'] == 'OVERALL' else None, axis=1
)

print (data_cleaned)

        Year        Location DataSource  \
0       2019        Arkansas      BRFSS   
1       2019           Idaho      BRFSS   
2       2019         Indiana      YRBSS   
3       2019            Iowa       NVSS   
4       2019            Iowa      BRFSS   
...      ...             ...        ...   
309210  2022  Virgin Islands      BRFSS   
309211  2022   West Virginia      BRFSS   
309212  2022       Wisconsin      BRFSS   
309213  2022         Vermont      BRFSS   
309214  2022      Washington      BRFSS   

                                        Topic  \
0                                    Diabetes   
1                                    Diabetes   
2                                       Sleep   
3                                      Asthma   
4                                      Asthma   
...                                       ...   
309210                                Tobacco   
309211  Chronic Obstructive Pulmonary Disease   
309212                           Immunizat

In [282]:
# Drop the original stratification columns
columns_to_drop = ['StratificationID1', 'StratificationCategoryID1', 'StratificationCategory1', 'Stratification1']
data_cleaned = data_cleaned.drop(columns=columns_to_drop)

print (data_cleaned)

        Year        Location DataSource  \
0       2019        Arkansas      BRFSS   
1       2019           Idaho      BRFSS   
2       2019         Indiana      YRBSS   
3       2019            Iowa       NVSS   
4       2019            Iowa      BRFSS   
...      ...             ...        ...   
309210  2022  Virgin Islands      BRFSS   
309211  2022   West Virginia      BRFSS   
309212  2022       Wisconsin      BRFSS   
309213  2022         Vermont      BRFSS   
309214  2022      Washington      BRFSS   

                                        Topic  \
0                                    Diabetes   
1                                    Diabetes   
2                                       Sleep   
3                                      Asthma   
4                                      Asthma   
...                                       ...   
309210                                Tobacco   
309211  Chronic Obstructive Pulmonary Disease   
309212                           Immunizat

In [283]:
print(data.columns)


Index(['YearStart', 'YearEnd', 'LocationAbbr', 'LocationDesc', 'DataSource',
       'Topic', 'Question', 'Response', 'DataValueUnit', 'DataValueType',
       'DataValue', 'DataValueAlt', 'DataValueFootnoteSymbol',
       'DataValueFootnote', 'LowConfidenceLimit', 'HighConfidenceLimit',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'StratificationCategory3', 'Stratification3',
       'Geolocation', 'LocationID', 'TopicID', 'QuestionID', 'ResponseID',
       'DataValueTypeID', 'StratificationCategoryID1', 'StratificationID1',
       'StratificationCategoryID2', 'StratificationID2',
       'StratificationCategoryID3', 'StratificationID3'],
      dtype='object')


In [284]:
# Group by State and count unique LocationIDs
unique_location_ids = data.groupby('LocationDesc')['LocationID'].nunique()
print(unique_location_ids)

LocationDesc
Alabama                 1
Alaska                  1
Arizona                 1
Arkansas                1
California              1
Colorado                1
Connecticut             1
Delaware                1
District of Columbia    1
Florida                 1
Georgia                 1
Guam                    1
Hawaii                  1
Idaho                   1
Illinois                1
Indiana                 1
Iowa                    1
Kansas                  1
Kentucky                1
Louisiana               1
Maine                   1
Maryland                1
Massachusetts           1
Michigan                1
Minnesota               1
Mississippi             1
Missouri                1
Montana                 1
Nebraska                1
Nevada                  1
New Hampshire           1
New Jersey              1
New Mexico              1
New York                1
North Carolina          1
North Dakota            1
Ohio                    1
Oklahoma                1

In [285]:
# Define the new column order
new_column_order = ['Year','Sex', 'Age', 'Education', 'Race/Ethnicity', 'Overall', 'Location','LocationID','Geolocation', 'Longitude', 'Latitude', 'Topic', 'TopicID', 'Question', 'QuestionID', 'Value', 'DataValueUnit', 'DataValueType', 'DataValueAlt', 'DataValueTypeID', 'LowConfidenceLimit', 'HighConfidenceLimit' ]

# Reorder the DataFrame
data_reordered = data_cleaned[new_column_order]

# Display the reordered DataFrame
print(data_reordered.head())

   Year   Sex        Age Education Race/Ethnicity  Overall  Location  \
0  2019  Male       None      None           None     None  Arkansas   
1  2019  Male       None      None           None     None     Idaho   
2  2019  None       None  Grade 12           None     None   Indiana   
3  2019  None       None      None           None  Overall      Iowa   
4  2019  None  Age 18-44      None           None     None      Iowa   

   LocationID                                    Geolocation   Longitude  ...  \
0           5   POINT (-92.27449074299966 34.74865012400045)  -92.274491  ...   
1          16  POINT (-114.3637300419997 43.682630005000476) -114.363730  ...   
2          18  POINT (-86.14996019399968 39.766910452000445)  -86.149960  ...   
3          19   POINT (-93.81649055599968 42.46940091300047)  -93.816491  ...   
4          19   POINT (-93.81649055599968 42.46940091300047)  -93.816491  ...   

   TopicID                                           Question QuestionID  \
0   

In [286]:
# Sort the dataset by TopicID
data_sorted = data_reordered.sort_values(by='TopicID')

# Verify the result
print(data_sorted[['TopicID', 'Topic']].head(20))  # Display the sorted topics


       TopicID    Topic
309214     ALC  Alcohol
225360     ALC  Alcohol
50538      ALC  Alcohol
225355     ALC  Alcohol
225348     ALC  Alcohol
225343     ALC  Alcohol
50553      ALC  Alcohol
225299     ALC  Alcohol
50572      ALC  Alcohol
225364     ALC  Alcohol
225286     ALC  Alcohol
225275     ALC  Alcohol
225271     ALC  Alcohol
225255     ALC  Alcohol
225232     ALC  Alcohol
225231     ALC  Alcohol
225224     ALC  Alcohol
50600      ALC  Alcohol
225222     ALC  Alcohol
225285     ALC  Alcohol


In [287]:
# Merge DataValue and DataValueUnit into a single column
data_sorted['ValueWithUnit'] = data_sorted['Value'].astype(str) + ' ' + data['DataValueUnit']

# View the updated DataFrame
print(data_sorted)


        Year     Sex   Age Education  \
309214  2022    Male  None      None   
225360  2021    None  None      None   
50538   2019    None  None      None   
225355  2021    None  None      None   
225348  2021    Male  None      None   
...      ...     ...   ...       ...   
245058  2021    None  None      None   
245077  2021    None  None      None   
101972  2019    None  None  Grade 12   
40397   2019  Female  None      None   
59291   2019    None  None      None   

                                    Race/Ethnicity Overall        Location  \
309214                                        None    None      Washington   
225360  Hawaiian or Pacific Islander, non-Hispanic    None  North Carolina   
50538                          Black, non-Hispanic    None         Indiana   
225355                   Multiracial, non-Hispanic    None      New Mexico   
225348                                        None    None         Montana   
...                                            ... 

In [288]:
# Find the unique values in the DataValueUnit column
unique_units = data_sorted['DataValueUnit'].unique()

# Print the unique units
print(unique_units)


['%' 'Number' 'cases per 100,000' 'gallons' 'per 100,000'
 'cases per 1,000,000' 'cases per 1,000' 'Years']


In [295]:
# Add a new column for each unique unit type, excluding "Number" in the formatted cells
unique_units = data_sorted['DataValueUnit'].unique()

# Create new columns
for unit in unique_units:
    data_sorted[unit] = data_sorted.apply(
        lambda row: f"{row['Value']} {unit}" if row['DataValueUnit'] == unit and unit != 'Number' else row['Value'] if row['DataValueUnit'] == unit else None,
        axis=1
    )

# View the updated DataFrame
print(data_sorted.head())


        Year   Sex   Age Education  \
309214  2022  Male  None      None   
225360  2021  None  None      None   
50538   2019  None  None      None   
225355  2021  None  None      None   
225348  2021  Male  None      None   

                                    Race/Ethnicity Overall        Location  \
309214                                        None    None      Washington   
225360  Hawaiian or Pacific Islander, non-Hispanic    None  North Carolina   
50538                          Black, non-Hispanic    None         Indiana   
225355                   Multiracial, non-Hispanic    None      New Mexico   
225348                                        None    None         Montana   

        LocationID                                    Geolocation   Longitude  \
309214          53  POINT (-120.47001078999972 47.52227862900048) -120.470011   
225360          37  POINT (-79.15925046299964 35.466220975000454)  -79.159250   
50538           18  POINT (-86.14996019399968 39.7669104520

In [296]:
# Save the DataFrame to the specified folder
file_path = r'C:\Users\info\Desktop\project3team11\Resources\cleaned_chronic_disease_data_ddw.csv'
data_sorted.to_csv(file_path, index=False)  

In [291]:
# Group by TopicID and calculate summary statistics for each disease
topic_grouped = data_reordered.groupby('Topic').agg({
    'Value': ['mean', 'min', 'max'],  # Example statistics
    'Location': 'count'  # Count of rows per topic
})

# Reset index for readability
topic_grouped.reset_index(inplace=True)

# Display the grouped data
print(topic_grouped)

                                              Topic        Value         \
                                                            mean    min   
0                                           Alcohol    84.916572    0.0   
1                                         Arthritis    31.859992    0.0   
2                                            Asthma    23.527662    0.0   
3                                            Cancer  1858.512419    0.7   
4                            Cardiovascular Disease  1860.673527    0.6   
5                            Chronic Kidney Disease  2702.197115  125.0   
6             Chronic Obstructive Pulmonary Disease  3084.580554    0.0   
7                   Cognitive Health and Caregiving    24.291830    4.2   
8                                          Diabetes   826.880313    0.4   
9                                        Disability    30.014520    6.8   
10                                    Health Status    12.278650    0.0   
11                       